In [2]:
#Dependencies 
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import csv

In [4]:
#import fort_state_zip.csv
fort_data =pd.read_csv('fort_state_zip.csv')
fort_data

,Name,State,Zip Code
0,Fort Novosel,AL,36362
1,Fort Huachuca,AZ,85613
2,Fort Irwin,CA,92310
3,Fort Carson,CO,80913
4,Fort Eisenhower,GA,30905
5,Fort Moore,GA,31905
6,Fort Stewart,GA,31313
7,Fort Leavenworth,KS,66027
8,Fort Riley,KS,66442
9,Fort Campbell,KY,42223


In [15]:
# Initialize Census with your API key
census_key = "cbc5dafcc66b3d11a52b9a7114bd19f59ae2d5da"
c = Census(census_key, year=2021)  # Try 2021 if 2023 is unsupported

# Define your list of zip codes as strings
fort_zips = fort_data['Zip Code'].astype(str).tolist()
fort_names = fort_data['Name'].tolist()

# Run Census Search to retrieve data on all zip codes
census_data = []
for zip_code in fort_zips:
    try:
        data = c.acs5.get(
            (
                "NAME",
                "B19013_001E",  # Household Income
                "B01003_001E",  # Population
                "B01002_001E",  # Median Age
                "B19301_001E",  # Per Capita Income
                "B17001_002E"   # Poverty Count
            ),
            {'for': f'zip code tabulation area:{zip_code}'}
        )
        census_data.extend(data)
    except CensusException as e:
        print(f"Error retrieving data for zip code {zip_code}: {e}")
        continue

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns={
        "B01003_001E": "Population",
        "B01002_001E": "Median Age",
        "B19013_001E": "Household Income",
        "B19301_001E": "Per Capita Income",
        "B17001_002E": "Poverty Count",
        "NAME": "Name",
        "zip code tabulation area": "Zipcode"
    }
)

census_pd['Name'] = fort_names
# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd


Number of rows in the DataFrame: 27


,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Zipcode
0,Fort Novosel,67574.0,5912.0,24.1,22206.0,691.0,36362
1,Fort Huachuca,88135.0,5248.0,22.7,25388.0,93.0,85613
2,Fort Irwin,58000.0,9100.0,22.8,23651.0,973.0,92310
3,Fort Carson,-666666666.0,5667.0,21.6,27354.0,0.0,80913
4,Fort Eisenhower,51333.0,7933.0,22.0,27368.0,452.0,30905
5,Fort Moore,60121.0,16973.0,22.6,22642.0,1664.0,31905
6,Fort Stewart,50526.0,41967.0,29.7,23524.0,7361.0,31313
7,Fort Leavenworth,99000.0,6698.0,24.8,27877.0,736.0,66027
8,Fort Riley,44219.0,16019.0,22.3,19177.0,2851.0,66442
9,Fort Campbell,51940.0,21028.0,21.8,20864.0,1233.0,42223


In [17]:
census_pd.replace(-666666666.0, pd.NA, inplace=True)
census_pd

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Zipcode
0,Fort Novosel,67574.0,5912.0,24.1,22206.0,691.0,36362
1,Fort Huachuca,88135.0,5248.0,22.7,25388.0,93.0,85613
2,Fort Irwin,58000.0,9100.0,22.8,23651.0,973.0,92310
3,Fort Carson,<NA>,5667.0,21.6,27354.0,0.0,80913
4,Fort Eisenhower,51333.0,7933.0,22.0,27368.0,452.0,30905
5,Fort Moore,60121.0,16973.0,22.6,22642.0,1664.0,31905
6,Fort Stewart,50526.0,41967.0,29.7,23524.0,7361.0,31313
7,Fort Leavenworth,99000.0,6698.0,24.8,27877.0,736.0,66027
8,Fort Riley,44219.0,16019.0,22.3,19177.0,2851.0,66442
9,Fort Campbell,51940.0,21028.0,21.8,20864.0,1233.0,42223


In [19]:
census_pd.to_excel('census_data.xlsx', index=True)